# Model Training

This notebook will cover the iterative process of training multiple models to find one best suited to our needs, and then further explore the possibilities of improving the selected model's performance.

### Import the necessary libraries

In [31]:
from deepod.models.time_series import DevNetTS, PReNetTS, DeepSADTS, DeepSVDDTS, DeepIsolationForestTS, AnomalyTransformer, COUTA, TcnED, TimesNet, TranAD, USAD
import pandas as pd
from sklearn.model_selection import train_test_split
from deepod.metrics import ts_metrics
from deepod.metrics import point_adjustment
import joblib
import torch

Assign the variable 'data_dir' with the location of the combined data file to be used for training.

In [2]:
data_dir= "D:\\anticheat\\data\\anomalyDataset.parquet"

In [3]:
df = pd.read_parquet(data_dir)

In [4]:
df.shape

(1670327, 65)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1670327 entries, 0 to 1670326
Data columns (total 65 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   tick                 1670327 non-null  float64
 1   seconds              1670327 non-null  float64
 2   clockTime            1670327 non-null  float64
 3   attackerSteamID      1670327 non-null  float64
 4   attackerX            1670327 non-null  float64
 5   attackerY            1670327 non-null  float64
 6   attackerZ            1670327 non-null  float64
 7   attackerViewY        1670327 non-null  float64
 8   victimViewY          1670327 non-null  float64
 9   zoomLevel            1670327 non-null  float64
 10  ctAlivePlayers       1670327 non-null  float64
 11  destroyTick          1670327 non-null  float64
 12  throwSeconds         1670327 non-null  float64
 13  destroySeconds       1670327 non-null  float64
 14  throwerX             1670327 non-null  float64
 15

In [6]:
df.describe()

,tick,seconds,clockTime,attackerSteamID,attackerX,attackerY,attackerZ,attackerViewY,victimViewY,zoomLevel,...,attackerStrafe,isSuicide,isHeadshot,isAlive,isBot,isAirborne,isStanding,isWalking,playerStrafe,label
count,1.670327e+06,1.670327e+06,1.670327e+06,1.670327e+06,1.670327e+06,1.670327e+06,1.670327e+06,1.670327e+06,1.670327e+06,1.670327e+06,...,1.670327e+06,1.670327e+06,1.670327e+06,1.670327e+06,1.670327e+06,1.670327e+06,1.670327e+06,1.670327e+06,1.670327e+06,1.670327e+06
mean,-9.619937e-04,9.495950e-04,-1.122814e-03,-1.116721e-02,1.415318e-03,3.983742e-04,-2.674495e-03,-4.928009e-03,-6.537796e-03,6.310782e-04,...,9.594888e-01,9.888094e-01,9.937791e-01,7.868729e-01,3.709328e-01,6.114671e-01,9.441163e-01,4.773766e-01,7.840375e-01,1.527084e-01
std,9.998579e-01,9.961785e-01,9.995313e-01,9.784140e-01,9.765762e-01,9.788238e-01,9.782046e-01,9.820481e-01,9.757139e-01,1.000711e+00,...,1.971550e-01,1.051922e-01,7.862723e-02,4.095169e-01,4.830546e-01,4.874169e-01,2.296970e-01,4.994881e-01,4.114885e-01,3.597063e-01
min,-2.084790e+00,-1.691118e+00,-8.263182e-01,-3.369892e-01,-1.168862e+01,-1.900945e+01,-1.098389e+01,-1.967590e-01,-1.927922e-01,-2.328260e-01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,-8.645809e-01,-7.491030e-01,-5.871486e-01,-2.550694e-01,-1.903572e-01,-1.694718e-01,-2.300961e-01,-1.396745e-01,-1.398519e-01,-1.655114e-01,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
50%,2.829388e-03,-2.061533e-01,-5.570595e-01,-2.443932e-01,1.471152e-01,6.844344e-02,-2.016530e-01,-1.318127e-01,-1.327716e-01,-1.424142e-01,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
75%,8.696227e-01,5.579936e-01,3.497812e-01,-2.329662e-01,1.899646e-01,1.544265e-01,2.275835e-01,-1.260387e-01,-1.264669e-01,-1.254552e-01,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
max,2.058158e+00,9.577903e+01,3.208749e+00,4.698008e+00,1.468193e+01,1.412862e+01,1.401423e+01,1.197718e+01,9.934007e+00,2.330945e+01,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


We will select a subset of the total data to quickly iterate over all available models and test their performances without changing the default hyperparameters, to set a baseline for improving performance.

In [7]:
sub_df = df.sample(20000)

*Note: Remember, our data has a temporal dependence, i.e., there is an order of events that makes sense. Hence, we cannot shuffle the data points and must pick a contiguous block of data as our subset.*

In [8]:
X = sub_df.drop(["label"], axis=1).values
y = sub_df["label"].values

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Set the device to GPU, if available.

In [35]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available.")
else:
    device = torch.device("cpu")
    print("GPU is not available.")

GPU is available.


# Unsupervised Models
***

In [37]:
model = DeepSVDDTS(device=device, network='LSTM', hidden_dims='64,100')
model.fit(X_train, y=None)

scores = model.decision_function(X_test)
anomalies = X_test[scores>0.5]

eval_metrics = ts_metrics(y_test, scores)
adj_eval_metrics = ts_metrics(y_test, point_adjustment(y_test, scores))
print("Results for DeepSAD:\n",
      f"auc: {adj_eval_metrics[1]:.2f}, average precision: {adj_eval_metrics[1]:.2f}, f1: {adj_eval_metrics[2]:.2f}, precision: {adj_eval_metrics[3]:.2f}, recall: {adj_eval_metrics[4]:.2f}")

Start Training...
ensemble size: 1
LSTMNet(
  (lstm): LSTM(64, 512, bias=False, batch_first=True)
  (fc): Linear(in_features=512, out_features=64, bias=True)
)
epoch  1, training loss: 0.260692, time: 1.0s
epoch 10, training loss: 0.042813, time: 0.3s
epoch 20, training loss: 0.019436, time: 0.3s
epoch 30, training loss: 0.012200, time: 0.3s
epoch 40, training loss: 0.008581, time: 0.3s
epoch 50, training loss: 0.006452, time: 0.3s
epoch 60, training loss: 0.005048, time: 0.3s
epoch 70, training loss: 0.004067, time: 0.3s
epoch 80, training loss: 0.003337, time: 0.3s
epoch 90, training loss: 0.002786, time: 0.3s
epoch100, training loss: 0.002349, time: 0.3s
Start Inference on the training data...


testing: 100%|██████████| 63/63 [00:00<00:00, 258.61it/s]

Results for DeepSAD:
 auc: 0.18, average precision: 0.18, f1: 0.29, precision: 0.19, recall: 0.64


In [24]:
model = AnomalyTransformer()
model.fit(X_train, y=None)

scores = model.decision_function(X_test)
anomalies = X_test[scores>0.5]

auc, ap, f1, p, r = ts_metrics(y_test, scores)
print("Results for DeepSAD:\n",
      f"auc: {auc:.2f}, average precision: {ap:.2f}, f1: {f1:.2f}, precision: {p:.2f}, recall: {r:.2f}")

epoch  1, training loss: 324.953355, time: 4.7s
epoch  2, training loss: 301.628813, time: 4.6s
epoch  3, training loss: 280.967560, time: 4.6s
epoch  4, training loss: 271.341669, time: 4.6s
epoch  5, training loss: 261.103043, time: 4.6s
epoch  6, training loss: 254.837644, time: 4.6s
epoch  7, training loss: 248.993177, time: 4.6s
epoch  8, training loss: 246.084679, time: 4.6s
epoch  9, training loss: 240.862193, time: 4.6s
epoch 10, training loss: 235.967497, time: 4.6s
Results for DeepSAD:
 auc: 0.49, average precision: 0.16, f1: 0.27, precision: 0.16, recall: 0.98


In [25]:
model = COUTA()
model.fit(X_train, y=None)

scores = model.decision_function(X_test)
anomalies = X_test[scores>0.5]

auc, ap, f1, p, r = ts_metrics(y_test, scores)
print("Results for DeepSAD:\n",
      f"auc: {auc:.2f}, average precision: {ap:.2f}, f1: {f1:.2f}, precision: {p:.2f}, recall: {r:.2f}")

|>>> epoch: 01  |   loss: 8.074183, loss_oc: 7.759422, val_loss: 2.373320
|>>> epoch: 02  |   loss: 2.215554, loss_oc: 2.114758, val_loss: 1.948444
|>>> epoch: 03  |   loss: 1.913427, loss_oc: 1.852996, val_loss: 1.799408
|>>> epoch: 04  |   loss: 1.791591, loss_oc: 1.736634, val_loss: 1.707144
|>>> epoch: 05  |   loss: 1.701746, loss_oc: 1.648504, val_loss: 1.621282
|>>> epoch: 06  |   loss: 1.610453, loss_oc: 1.557907, val_loss: 1.522914
|>>> epoch: 07  |   loss: 1.496665, loss_oc: 1.445061, val_loss: 1.388249
|>>> epoch: 08  |   loss: 1.337327, loss_oc: 1.287193, val_loss: 1.196527
|>>> epoch: 09  |   loss: 1.103822, loss_oc: 1.055313, val_loss: 0.918698
|>>> epoch: 10  |   loss: 0.584027, loss_oc: 0.537192, val_loss: 0.345472
|>>> epoch: 11  |   loss: 0.329923, loss_oc: 0.284579, val_loss: 0.249893
|>>> epoch: 12  |   loss: 0.266456, loss_oc: 0.221914, val_loss: 0.206849
|>>> epoch: 13  |   loss: 0.230316, loss_oc: 0.187791, val_loss: 0.177976
|>>> epoch: 14  |   loss: 0.204742, lo

In [26]:
model = TcnED()
model.fit(X_train, y=None)

scores = model.decision_function(X_test)
anomalies = X_test[scores>0.5]

auc, ap, f1, p, r = ts_metrics(y_test, scores)
print("Results for DeepSAD:\n",
      f"auc: {auc:.2f}, average precision: {ap:.2f}, f1: {f1:.2f}, precision: {p:.2f}, recall: {r:.2f}")

Start Training...
ensemble size: 1
TcnAE(
  (enc_0): TcnResidualBlock(
    (conv1): Conv1d(64, 32, kernel_size=(3,), stride=(1,), padding=(2,), bias=False)
    (chomp1): Chomp1d()
    (act1): ReLU()
    (dropout1): Dropout(p=0.2, inplace=False)
    (conv2): Conv1d(32, 32, kernel_size=(3,), stride=(1,), padding=(2,), bias=False)
    (chomp2): Chomp1d()
    (act2): ReLU()
    (dropout2): Dropout(p=0.2, inplace=False)
    (net): Sequential(
      (0): Conv1d(64, 32, kernel_size=(3,), stride=(1,), padding=(2,), bias=False)
      (1): Chomp1d()
      (2): ReLU()
      (3): Dropout(p=0.2, inplace=False)
      (4): Conv1d(32, 32, kernel_size=(3,), stride=(1,), padding=(2,), bias=False)
      (5): Chomp1d()
      (6): ReLU()
      (7): Dropout(p=0.2, inplace=False)
    )
    (downsample): Conv1d(64, 32, kernel_size=(1,), stride=(1,))
    (act): ReLU()
  )
  (enc_1): TcnResidualBlock(
    (conv1): Conv1d(32, 32, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(2,), bias=False)
    (chomp1

testing: 100%|██████████| 122/122 [00:00<00:00, 136.45it/s]


Results for DeepSAD:
 auc: 0.47, average precision: 0.14, f1: 0.27, precision: 0.16, recall: 0.97


In [27]:
model = TimesNet()
model.fit(X_train, y=None)

scores = model.decision_function(X_test)
anomalies = X_test[scores>0.5]

auc, ap, f1, p, r = ts_metrics(y_test, scores)
print("Results for DeepSAD:\n",
      f"auc: {auc:.2f}, average precision: {ap:.2f}, f1: {f1:.2f}, precision: {p:.2f}, recall: {r:.2f}")

epoch  1, training loss: 23.219847, time: 15.2s
epoch  2, training loss: 20.660067, time: 11.0s
epoch  3, training loss: 19.242502, time: 11.1s
epoch  4, training loss: 20.080682, time: 11.3s
epoch  5, training loss: 16.785863, time: 12.7s
epoch  6, training loss: 14.225079, time: 13.8s
epoch  7, training loss: 14.556054, time: 13.7s
epoch  8, training loss: 13.173334, time: 14.2s
epoch  9, training loss: 13.831975, time: 16.5s
epoch 10, training loss: 13.580570, time: 16.9s
Results for DeepSAD:
 auc: 0.50, average precision: 0.16, f1: 0.27, precision: 0.16, recall: 0.95


In [28]:
model = TranAD()
model.fit(X_train, y=None)

scores = model.decision_function(X_test)
anomalies = X_test[scores>0.5]

auc, ap, f1, p, r = ts_metrics(y_test, scores)
print("Results for DeepSAD:\n",
      f"auc: {auc:.2f}, average precision: {ap:.2f}, f1: {f1:.2f}, precision: {p:.2f}, recall: {r:.2f}")

Epoch 1,	 L1 = 396.11858853426844
Epoch 2,	 L1 = 396.65878989479756
Epoch 3,	 L1 = 392.1478701504794
Epoch 4,	 L1 = 395.4637520530007
Epoch 5,	 L1 = 396.52113203568894
Results for DeepSAD:
 auc: 0.50, average precision: 0.16, f1: 0.27, precision: 0.16, recall: 0.98


In [29]:
model = DeepIsolationForestTS()
model.fit(X_train, y=None)

scores = model.decision_function(X_test)
anomalies = X_test[scores>0.5]

auc, ap, f1, p, r = ts_metrics(y_test, scores)
print("Results for DeepSAD:\n",
      f"auc: {auc:.2f}, average precision: {ap:.2f}, f1: {f1:.2f}, precision: {p:.2f}, recall: {r:.2f}")

Start Training...


  0%|          | 0/50 [00:00<?, ?it/s]d:\anticheat\anticheat_env\lib\site-packages\deepod\core\networks\network_utility.py:23: UserWarning: use the first hidden num, the rest hidden numbers are deprecated
  warnings.warn('use the first hidden num, '
  2%|▏         | 1/50 [00:01<01:18,  1.59s/it]d:\anticheat\anticheat_env\lib\site-packages\deepod\core\networks\network_utility.py:23: UserWarning: use the first hidden num, the rest hidden numbers are deprecated
  warnings.warn('use the first hidden num, '
  4%|▍         | 2/50 [00:03<01:12,  1.50s/it]d:\anticheat\anticheat_env\lib\site-packages\deepod\core\networks\network_utility.py:23: UserWarning: use the first hidden num, the rest hidden numbers are deprecated
  warnings.warn('use the first hidden num, '
  6%|▌         | 3/50 [00:04<01:09,  1.47s/it]d:\anticheat\anticheat_env\lib\site-packages\deepod\core\networks\network_utility.py:23: UserWarning: use the first hidden num, the rest hidden numbers are deprecated
  warnings.warn('use 

Start Inference on the training data...
Start Inference...


100%|██████████| 50/50 [01:43<00:00,  2.07s/it]


Start Inference...


100%|██████████| 50/50 [00:22<00:00,  2.21it/s]

Results for DeepSAD:
 auc: 0.50, average precision: 0.16, f1: 0.27, precision: 0.16, recall: 0.98


In [ ]:
model = USAD()
model.fit(X_train, y=None)

scores = model.decision_function(X_test)
anomalies = X_test[scores>0.5]

auc, ap, f1, p, r = ts_metrics(y_test, scores)
print("Results for DeepSAD:\n",
      f"auc: {auc:.2f}, average precision: {ap:.2f}, f1: {f1:.2f}, precision: {p:.2f}, recall: {r:.2f}")

# Weakly-supervised models
***

In [ ]:
model = DevNetTS(seq_len=50, )
print("X:", X_train.shape,"y:", y_train.shape)
model.fit(X_train, y_train)

scores = model.decision_function(X_test)
anomalies = X_test[scores>0.5]

auc, ap, f1, p, r = ts_metrics(y_test, scores)
print("Results for DeepSAD:\n",
      f"auc: {auc:.2f}, average precision: {ap:.2f}, f1: {f1:.2f}, precision: {p:.2f}, recall: {r:.2f}")

In [ ]:
model = DeepSADTS(batch_size=100, lr=0.001, rep_dim=128, hidden_dims='100,50', act='ReLU', bias=False, epoch_steps=-1)
model.fit(X_train, y_train)

scores = model.decision_function(X_test)
anomalies = X_test[scores>0.5]

auc, ap, f1, p, r = ts_metrics(y_test, scores)
print("Results for DeepSAD:\n",
      f"auc: {auc:.2f}, average precision: {ap:.2f}, f1: {f1:.2f}, precision: {p:.2f}, recall: {r:.2f}")

In [ ]:
model = PReNetTS(batch_size=100, lr=0.001, rep_dim=128, hidden_dims='100,50', act='ReLU', bias=False, epoch_steps=-1)
model.fit(X_train, y_train)

scores = model.decision_function(X_test)
anomalies = X_test[scores>0.5]

auc, ap, f1, p, r = ts_metrics(y_test, scores)
print("Results for DeepSAD:\n",
      f"auc: {auc:.2f}, average precision: {ap:.2f}, f1: {f1:.2f}, precision: {p:.2f}, recall: {r:.2f}")